In [1]:
import cv2
import numpy as np
import matplotlib.pylab as plt
import keras
from keras.layers import \
    Dense, Dropout, Activation, Flatten, \
    Convolution2D, MaxPooling2D, Lambda
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os


Using TensorFlow backend.


In [88]:
import glob
from imgaug import augmenters as iaa
from keras.utils import np_utils

In [94]:
# model
model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(64,64,3)))
model.add(Convolution2D(8, 3, 3, activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(12, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
model.add(Convolution2D(16, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
def aug(imgs, max_num, seq):
    n = max_num // len(imgs) - 1
    b = max_num % len(imgs)
    res = imgs
    for _ in range(n):
        aug_imgs = seq.augment_images(imgs)
        res = res + aug_imgs
    for i in range(b):
        img = seq.augment_images(imgs[i])
        res.append(img)
    return res


In [48]:
red = [cv2.imread(f) for f in glob.glob('*_state_0.jpg')]
yellow = [cv2.imread(f) for f in glob.glob('*_state_1.jpg')]
green = [cv2.imread(f) for f in glob.glob('*_state_2.jpg')]
unknown = [cv2.imread(f) for f in glob.glob('*_state_4.jpg')]

In [68]:
print len(red)
print len(yellow_aug)
print len(green_aug)
print len(unknown_aug)

456
456
456
456


In [69]:
top_len = len(red)

In [65]:
seq = iaa.Sequential([
    iaa.Fliplr(.5),
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
    iaa.GaussianBlur(sigma=(0, 1.0))
])
yellow_aug = aug(yellow, len(red), seq)

In [66]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
    iaa.GaussianBlur(sigma=(0.,1.0))
])
green_aug = aug(green, len(red), seq)

In [67]:
seq = iaa.Sequential([
    iaa.Fliplr(.5),
    iaa.AdditiveGaussianNoise(0, scale=(0,255), per_channel=0.8),
    iaa.GaussianBlur(sigma=(0, 3.0))
])
unknown_aug = aug(unknown, len(red), seq)

In [133]:
xs = red + yellow_aug + green_aug + unknown_aug
ys = [0]*top_len + [1]*top_len + [2]*top_len + [3]*top_len

xs_tr = np.array(xs)
ys_tr = np_utils.to_categorical(ys, 4)

xs_tr, ys_tr = shuffle(xs_tr, ys_tr)

In [136]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,verbose=0)
]
model.fit(x=xs_tr, y=ys_tr, batch_size=64, nb_epoch=1, validation_split=0.1, callbacks=callbacks)
model.save('tl_model.h5')

Train on 1641 samples, validate on 183 samples
Epoch 1/1
1641/1641 [==============================] - 4s - loss: 0.0118 - acc: 0.9957 - val_loss: 0.0078 - val_acc: 1.0000
